In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, ReLU, Add, Dense, Flatten, Concatenate
from keras.optimizers import Adam, RMSprop
from keras.utils import load_img, img_to_array, to_categorical, plot_model
from keras.regularizers import L1L2

import numpy as np
import pandas as pd
import os


In [2]:
DIR_TRAIN = "./train/"
DIR_TEST = "./test/"

PATH_TRAIN = "./train.csv"
PATH_TEST = "./test.csv"

In [3]:
data = pd.read_csv(PATH_TRAIN, delimiter=',', header=0)
data.head()

num_classes = int(data['class'].max()) + 1

In [4]:
import shutil

DIR_VAL = "./val/"

def data_for_dataset():
    
    for i in range(num_classes):
        dir = os.path.join(DIR_TRAIN, str(i))
        if not os.path.exists(dir):
            os.mkdir(dir)
        for f in data[data['class']==i].values:
            fn = os.path.join(dir, f[0])
            if not os.path.exists(fn):
                shutil.move(os.path.join(DIR_TRAIN, f[0]), fn)
    
    if not os.path.exists(DIR_VAL):
        os.mkdir(DIR_VAL)
    for i in range(num_classes):
        dir = os.path.join(DIR_VAL, str(i))
        if not os.path.exists(dir):
            os.mkdir(dir)
        ls = sorted(os.listdir(os.path.join(DIR_TRAIN, str(i))))
        for j in range(len(ls)//10):
            ffrom = os.path.join(DIR_TRAIN, str(i), ls[j])
            fto = os.path.join(DIR_VAL, str(i), ls[j])
            if os.path.exists(ffrom) and not os.path.exists(fto):
                shutil.move(ffrom, fto)

#data_for_dataset()


In [5]:
shape = (128, 128)
xtrain = np.array([img_to_array(load_img(os.path.join(DIR_TRAIN, s), color_mode='rgb', target_size=shape, keep_aspect_ratio=True)) for s in data.ID_img]) / 255.
ytrain = np.array([to_categorical(s, num_classes=num_classes) for s in data['class']])

#xtrain, xtest, ytrain, ytest = train_test_split(xtrain, ytrain, test_size=.1)
#print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)
print(xtrain.shape, ytrain.shape)

(4990, 128, 128, 3) (4990, 8)


In [6]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(featurewise_center=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.1)
datagen.fit(xtrain)

In [7]:
def model1():
    l1l2 = L1L2(l1=1e-5, l2=1e-4)
    
    inp = Input(shape=(128, 128, 3))

    x = Conv2D(8, 3, padding='valid', kernel_regularizer=l1l2)(inp)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    for k in [16, 32, 64, 128, 256]:
        for i in range(3):
            y = Conv2D(k, 3, padding='same', kernel_regularizer=l1l2)(x)
            y = BatchNormalization()(y)
            y = ReLU()(y)
            y = Conv2D(k, 3, padding='same', kernel_regularizer=l1l2)(y)
            if i == 2:
                y = MaxPooling2D(2)(y)
            
            x = Conv2D(k, 1, padding='same', kernel_regularizer=l1l2)(x)
            if i == 2:
                x = MaxPooling2D(2)(x)
            
            x = Add()([x,y])
            x = BatchNormalization()(x)
            x = ReLU()(x)
    
    x = Conv2D(512, 3, padding='valid', kernel_regularizer=l1l2)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Flatten()(x)

    for i in range(2):
        x = Dense(4096, kernel_regularizer=l1l2)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inp, x)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics='accuracy')

    return model, './ulite.h5'

model, path_weights = model1()
#plot_model(model, show_layer_names=True, show_shapes=True, dpi=64)


In [8]:
def model2():
    inp = Input(shape=(128, 128, 3))
    x = BatchNormalization()(inp)

    for k in [64, 128]:
        #x = Conv2D(k, 3, padding='valid')(x)
        #x = BatchNormalization()(x)
        #x = ReLU()(x)
        x = Conv2D(k, 3, strides=2, padding='valid')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    a = []
    for i in range(16):
        y = Conv2D(16, 9, padding='valid')(x)
        y = BatchNormalization()(y)
        y = ReLU()(y)
        y = Conv2D(16, 9, strides=2, padding='valid')(y)
        a.append(y)
    
    x = Concatenate()(a)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(512, 3, padding='valid')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    #x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inp, x)
    model.compile(optimizer=RMSprop(learning_rate=1e-4), loss='categorical_crossentropy', metrics='accuracy')

    return model, './ulite2.h5'

#plot_model(model, show_layer_names=True, show_shapes=True, dpi=64)


In [9]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

model, path_weights = model1()

es = EarlyStopping(patience=10, verbose=1, restore_best_weights=True)
mcp = ModelCheckpoint(path_weights, mode='min', save_best_only=True, save_weights_only=True)
rop = ReduceLROnPlateau(patience=8, min_lr=1e-10, verbose=1)


In [10]:
if not os.path.exists(path_weights):
    model.load_weights(path_weights)
history = model.fit(datagen.flow(xtrain, ytrain, batch_size=100, subset='training'), epochs=100, verbose=1, validation_data=datagen.flow(xtrain, ytrain, batch_size=100, subset='validation'), callbacks=[es, mcp, rop])

Epoch 1/100
45/45 [==============================] - 28s 475ms/step - loss: 6.9243 - accuracy: 0.2928 - val_loss: 6.7165 - val_accuracy: 0.2224 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - 19s 421ms/step - loss: 6.7026 - accuracy: 0.3547 - val_loss: 6.6660 - val_accuracy: 0.3026 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - 19s 423ms/step - loss: 6.4811 - accuracy: 0.4195 - val_loss: 6.7415 - val_accuracy: 0.3026 - lr: 1.0000e-04
Epoch 4/100
45/45 [==============================] - 19s 425ms/step - loss: 6.3196 - accuracy: 0.4743 - val_loss: 6.8115 - val_accuracy: 0.3026 - lr: 1.0000e-04
Epoch 5/100
45/45 [==============================] - 19s 427ms/step - loss: 6.2080 - accuracy: 0.5181 - val_loss: 6.9079 - val_accuracy: 0.3026 - lr: 1.0000e-04
Epoch 6/100
45/45 [==============================] - 19s 429ms/step - loss: 6.0728 - accuracy: 0.5469 - val_loss: 7.0624 - val_accuracy: 0.3026 - lr: 1.0000e-04
Epoch 7/100
13/45 [=======>.......

In [ ]:
#model.load_weights(path_weights)
model.evaluate(datagen.flow(xtrain, ytrain, batch_size=20, subset='validation'))

In [ ]:
data = pd.read_csv(PATH_TEST, delimiter=',', header=0)
data.head()

In [ ]:
xval = np.array([img_to_array(load_img(os.path.join(DIR_TEST, s), color_mode='rgb', target_size=shape, keep_aspect_ratio=True)) for s in data.ID_img]) / 255.

In [ ]:
pred = model.predict(xval)
pred = [np.argmax(i) for i in pred]

In [ ]:
del(data['class'])
data['class'] = pred
data.head()

In [ ]:
data.to_csv('./submit.csv', index=False)